# Installation

In [1]:
!pip install transformers datasets

# Imports

In [2]:
pip uninstall -y numpy tensorflow gensim scipy


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
Found existing installation: scipy 1.13.1
Uninstalling scipy-1.13.1:
  Successfully uninstalled scipy-1.13.1


In [3]:
pip install numpy tensorflow gensim scipy tensorflow-probability

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached tensorflow-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached numpy-2.1.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached tensorflow-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (644.9 MB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux20

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, roc_curve### metrics

import pathlib
import io
import os
import re
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files


AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [2]:
 from datasets import load_dataset
from transformers import (BertTokenizerFast,TFBertTokenizer,BertTokenizer,RobertaTokenizerFast,
                          DataCollatorWithPadding,TFRobertaForSequenceClassification,TFBertForSequenceClassification,
                          TFBertModel,create_optimizer)

RuntimeError: Failed to import transformers.models.bert.tokenization_bert_tf because of the following error (look up to see its traceback):
/usr/local/lib/python3.11/dist-packages/tensorflow_text/python/ops/_regex_split_ops.so: undefined symbol: _ZN3tsl5mutexC1Ev

In [ ]:
BATCH_SIZE=8

# Data Preparation for Bert Model

In [ ]:
dataset_id = 'imdb'
dataset = load_dataset(dataset_id)

In [ ]:
dataset

In [ ]:
dataset['train'][0]

## Bert Model

In [ ]:
model_id="bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_id)

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"], padding=True, truncation=True)

In [ ]:
toknized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
toknized_dataset

In [ ]:
tf_train_dataset = toknized_dataset["train"].to_tf_dataset(
    columns=['input_ids','token_type_ids','attention_mask','label'],
    shuffle=True,
    batch_size=BATCH_SIZE,
)

In [ ]:
tf_train_dataset

In [ ]:
tf_val_dataset = toknized_dataset["test"].to_tf_dataset(
    columns=['input_ids','token_type_ids','attention_mask','label'],
    shuffle=True,
    batch_size=BATCH_SIZE,
)

In [ ]:
def swap_positions(dataset):
    return {
        'input_ids': dataset['input_ids'],
        'token_type_ids': dataset['token_type_ids'],
        'attention_mask': dataset['attention_mask']
    }, dataset['label']


In [ ]:
tf_train_dataset = tf_train_dataset.map(swap_positions).prefetch(tf.data.AUTOTUNE)
tf_val_dataset = tf_val_dataset.map(swap_positions).prefetch(tf.data.AUTOTUNE)

In [ ]:
for i in tf_train_dataset.take(1):
  print(i)

In [ ]:
tf_val_dataset

# Modeling

## Based on TFBertModel

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)
model.summary()

# Training

In [ ]:
num_epochs = 3
batches_per_epoch = len(toknized_dataset["train"])// BATCH_SIZE
total_train_steps = int(batches_per_epoch*num_epochs)

In [ ]:
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model.compile(optimizer=optimizer,metrics=['accuracy'])

In [ ]:
history=model.fit(tf_train_dataset,validation_data=tf_val_dataset, epochs=num_epochs)